# jupyterlite_pyodide_kernel

> [`jupyterlite_pyodide_kernel`](https://github.com/jupyterlite/pyodide-kernel) provides the IPython/ipykernel interactive computing experience in the JupyterLite client, powered by the `pyodide` Python distribution.

In [ ]:
GH_ORG = "jupyterlite"
NAME = "jupyterlite_pyodide_kernel"
JS_NS = "@jupyterlite"
URL = f"https://github.com/{GH_ORG}/pyodide-kernel"

In [ ]:
import importnb

with importnb.Notebook():
    from jupyak.tasks import __jupyterlab, __jupyterlite, __lumino
    from jupyak.tasks import _well_known as W
    from jupyak.tasks import _yak as Y

## schema

> use the hacked labextension script to bypass location/version checks

In [ ]:
pk = "packages/pyodide-kernel"
pke = "packages/pyodide-kernel-extension"
pk_tsbi = f"{pk}/{W.TSCONFIG_BUILDINFO}"
pke_tsbi = f"{pke}/{W.TSCONFIG_BUILDINFO}"
pk_pypi_all = f"{pk}/pypi/all.json"
JS_DIST_TASKS = {
    f"./{pk}/": [
        {
            "name": "build:py:wheels",
            "actions": [["yarn", "build:py"]],
            "file_dep": [
                f"{pk}/py/**/pyproject.toml",
                f"{pk}/py/**/*.py",
            ],
            "targets": [pk_pypi_all],
            "needs_pth": [
                "jupyterlite_core",
                NAME,
            ],
        },
        {
            "name": "build:js",
            "actions": [["yarn", "build:prod"]],
            "file_dep": [pk_pypi_all, f"{pk}/src/**/*.ts"],
            "targets": [pk_tsbi],
        },
    ],
    f"./{pke}/": [
        {
            "name": "build:lib",
            "actions": [["yarn", "build:lib"]],
            "file_dep": [f"{pke}/src/**/*.ts", pk_tsbi],
            "targets": [pke_tsbi],
        },
    ],
}

In [ ]:
py_ext_pj = f"jupyterlite_pyodide_kernel/labextension/{W.PACKAGE_JSON}"
LAB_EXT = {f"./{pke}/": {"file_dep": [pk_pypi_all, pke_tsbi], "targets": [py_ext_pj]}}

In [ ]:
DEFAULTS = {
    "name": NAME,
    "github": {"url": URL},
    "js": {
        "dependencies": [
            __lumino.NAME,
            __jupyterlab.NAME,
            __jupyterlite.NAME,
        ],
        "tasks": JS_DIST_TASKS,
    },
    "py": {
        "dependencies": [
            __jupyterlab.NAME,
            __jupyterlite.NAME,
        ],
        "lab_extensions": LAB_EXT,
    },
    "lite": {"needs_pth": [NAME]},
}

In [ ]:
@Y.Yak.repo(
    NAME,
    tuple(DEFAULTS["py"]["dependencies"] + DEFAULTS["js"]["dependencies"]),
)
def default_factory(yak: Y.Yak):
    return DEFAULTS